In [1]:
import pandas as pd
from utils import get_data_with_filtered_columns

# Preparing data 

In [2]:
X, y, final_columns = get_data_with_filtered_columns(top_columns=10, correlation_threshold=0.8)
X

/home/uxeruses/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,48,64,105,241,282,323,338,424,442,453
0,440,648,181,434,492,476,628,496,568,471
1,499,488,431,551,491,463,528,457,463,311
2,460,485,593,474,485,486,431,474,503,606
3,529,415,698,569,470,458,377,501,447,545
4,429,387,451,538,497,485,509,504,536,426
...,...,...,...,...,...,...,...,...,...,...
1995,505,482,541,535,526,466,419,502,509,570
1996,448,311,696,590,483,478,348,475,421,423
1997,544,540,752,463,474,471,406,480,428,643
1998,490,543,524,443,478,479,473,469,563,691


In [5]:
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, stratify=y)


# Simple voting classifier

In [13]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

# List of single models
model1 = LogisticRegression(random_state=42)
model2 = DecisionTreeClassifier(random_state=42)
model3 = SVC(probability=True, random_state=42)
model4 = RandomForestClassifier(random_state=42)
model5 = KNeighborsClassifier()
model6 = GaussianNB()
model7 = MLPClassifier(random_state=42)
model8 = GradientBoostingClassifier(random_state=42)

# Create an ensemble for classification
ensemble_model = VotingClassifier(estimators=[
    ('logistic', model1),
    ('decision_tree', model2),
    ('svm', model3),
    ('random_forest', model4),
    ('k_neighbors', model5),
    ('naive_bayes', model6),
    ('mlp', model7),
    ('gradient_boosting', model8)
], voting='soft')



In [14]:
# Fitting the ensembled model
ensemble_model.fit(X_train, y_train)

/home/uxeruses/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


VotingClassifier(estimators=[('logistic', LogisticRegression(random_state=42)),
                             ('decision_tree',
                              DecisionTreeClassifier(random_state=42)),
                             ('svm', SVC(probability=True, random_state=42)),
                             ('random_forest',
                              RandomForestClassifier(random_state=42)),
                             ('k_neighbors', KNeighborsClassifier()),
                             ('naive_bayes', GaussianNB()),
                             ('mlp', MLPClassifier(random_state=42)),
                             ('gradient_boosting',
                              GradientBoostingClassifier(random_state=42))],
                 voting='soft')

In [15]:
# Predictions on test set
predictions = ensemble_model.predict(X_test)

# Accuracy score of trained model on test set
accuracy = accuracy_score(y_test, predictions)
print(f'Ensemble Model Accuracy: {accuracy:.4f}')

Ensemble Model Accuracy: 0.8240


# Voting classifier with tuning hyperparameters

In [18]:
# Create a dictionary of hyperparameters for each model
param_distributions = {
    'logistic__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'decision_tree__max_depth': [None, 10, 20, 30],
    'svm__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'random_forest__n_estimators': [50, 100, 200],
    'random_forest__max_depth': [None, 10, 20, 30],
    'k_neighbors__n_neighbors': [3, 5, 7],
    'mlp__hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'mlp__alpha': [0.0001, 0.001, 0.01],
    'gradient_boosting__n_estimators': [50, 100, 200],
    'gradient_boosting__learning_rate': [0.01, 0.1, 0.2],
}

# Perform RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(ensemble_model, param_distributions, n_iter=500, scoring='accuracy', cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

# Get the best hyperparameters and fit the final model
best_params = random_search.best_params_
best_model = random_search.best_estimator_


/home/uxeruses/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/uxeruses/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/uxeruses/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

In [19]:
# Predictions on test set
predictions = best_model.predict(X_test)

# Accuracy score of trained model on test set
accuracy = accuracy_score(y_test, predictions)
print(f'Tuned Ensemble Model Accuracy: {accuracy:.4f}')

# Best parameters
print(f'Best Hyperparameters: {best_params}')

Tuned Ensemble Model Accuracy: 0.8460
Best Hyperparameters: {'svm__C': 100, 'random_forest__n_estimators': 200, 'random_forest__max_depth': 30, 'mlp__hidden_layer_sizes': (100,), 'mlp__alpha': 0.001, 'logistic__C': 0.01, 'k_neighbors__n_neighbors': 3, 'gradient_boosting__n_estimators': 50, 'gradient_boosting__learning_rate': 0.01, 'decision_tree__max_depth': 10}


# Predicting values for test data

In [21]:
x_final = pd.read_csv("data/artificial_test.data", sep=' ', header=None)
x_final = x_final.drop(x_final.columns[-1], axis=1)

x_final = x_final[final_columns]

In [22]:
y_final_proba = best_model.predict_proba(x_final)
y_final_aml = y_final_proba[:, 1]

In [23]:
pd.DataFrame(y_final_proba).head()

,0,1
0,0.680142,0.319858
1,0.817500,0.182500
2,0.518956,0.481044
3,0.337427,0.662573
4,0.632026,0.367974


In [25]:
import numpy as np

np.savetxt('data/313180_313282_artifical_model_prediction.txt', y_final_aml, delimiter=',', fmt='%s')